##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial de componentes de funciones de Python para TFX


Nota: Recomendamos ejecutar este tutorial en un bloc de notas de Colab, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/python_function_component"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/python_function_component.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

Este bloc de notas contiene ejemplos sobre cómo crear y ejecutar componentes de funciones de Python dentro del InteractiveContext de TFX y en una canalización de TFX orquestada localmente.

Para obtener más contexto e información, consulte la página de [componentes de funciones personalizadas de Python](https://www.tensorflow.org/tfx/guide/custom_function_component) en el sitio de documentación de TFX.

## Preparación

Primero instalaremos TFX e importaremos los módulos necesarios. TFX requiere Python 3.

### Comprobación de la versión de Python del sistema


In [ ]:
import sys
sys.version

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instalación de TFX

**Nota: En Google Colab, debido a las actualizaciones de paquetes, la primera vez que ejecuta esta celda debe reiniciar el tiempo de ejecución (Tiempo de ejecución &gt; Reiniciar tiempo de ejecución...).**

In [ ]:
!pip install tfx

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

## ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe reiniciar el tiempo de ejecución (Tiempo de ejecución &gt; Reiniciar tiempo de ejecución...). Esto se debe a la forma en que Colab carga los paquetes.

### Importación de paquetes

Importamos TFX y comprobamos su versión.

In [ ]:
# Check version
from tfx import v1 as tfx
tfx.__version__

## Componentes personalizados de funciones de Python

En esta sección, crearemos componentes a partir de funciones de Python. No trabajaremos con ningún problema real de aprendizaje automático: estas funciones simples solo se usan para ilustrar el proceso de desarrollo de componentes de funciones de Python.

Consulte la [guía de componentes basados en funciones de Python](https://www.tensorflow.org/tfx/guide/custom_function_component) para acceder a más documentación al respecto.

### Cómo crear componentes personalizados de Python

Para empezar, se escribe una función que genera algunos datos ficticios. Esto se escribe en su propio archivo de módulo de Python.

In [ ]:
%%writefile my_generator.py

import os
import tensorflow as tf  # Used for writing files.

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset

@tfx.dsl.components.component
def MyGenerator(data: tfx.dsl.components.OutputArtifact[Dataset]):
  """Create a file with dummy data in the output artifact."""
  with tf.io.gfile.GFile(os.path.join(data.uri, 'data_file.txt'), 'w') as f:
    f.write('Dummy data')

  # Set metadata and ensure that it gets passed to downstream components.
  data.set_string_custom_property('my_custom_field', 'my_custom_value')

A continuación, se escribe un segundo componente que usa los datos ficticios producidos. Simplemente calcularemos el hash de los datos y los devolveremos.

In [ ]:
%%writefile my_consumer.py

import hashlib
import os
import tensorflow as tf

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset
from tfx.types.standard_artifacts import String

@tfx.dsl.components.component
def MyConsumer(data: tfx.dsl.components.InputArtifact[Dataset],
               hash: tfx.dsl.components.OutputArtifact[String],
               algorithm: tfx.dsl.components.Parameter[str] = 'sha256'):
  """Reads the contents of data and calculate."""
  with tf.io.gfile.GFile(
      os.path.join(data.uri, 'data_file.txt'), 'r') as f:
    contents = f.read()
  h = hashlib.new(algorithm)
  h.update(tf.compat.as_bytes(contents))
  hash.value = h.hexdigest()

  # Read a custom property from the input artifact and set to the output.
  custom_value = data.get_string_custom_property('my_custom_field')
  hash.set_string_custom_property('input_custom_field', custom_value)

### Cómo ejecutar en el bloc de notas con InteractiveContext

Ahora, demostraremos el uso de nuestros nuevos componentes en TFX InteractiveContext.

Para obtener más información sobre lo que puede hacer con el componente InteractiveContext del bloc de notas TFX, consulte el [Tutorial del componente TFX Keras](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) del bloc de notas.

In [ ]:
from my_generator import MyGenerator
from my_consumer import MyConsumer

#### Cómo construir InteractiveContext

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

#### Cómo ejecutar el componente de forma interactiva con `context.run()`

A continuación, ejecutamos nuestros componentes de forma interactiva dentro del bloc de notas con ayuda de `context.run()`. Nuestro componente de consumo usa las salidas del componente generador.

In [ ]:
generator = MyGenerator()
context.run(generator)

In [ ]:
consumer = MyConsumer(
    data=generator.outputs['data'],
    algorithm='md5')
context.run(consumer)

Tras la ejecución, podemos inspeccionar el contenido del artefacto de salida "hash" del componente consumidor en el disco.

In [ ]:
!tail -v {consumer.outputs['hash'].get()[0].uri}

¡Eso es todo! ¡Ya escribió y ejecutó sus propios componentes personalizados!

### Cómo escribir una definición de canalización

A continuación, crearemos una canalización con estos mismos componentes. Si bien el uso de `InteractiveContext` dentro de un bloc de notas funciona bien durante la experimentación, definir una canalización le permite implementar su canalización en ejecutores locales o remotos para usar en producción.

Aquí, demostraremos el uso del LocalDagRunner cuando se ejecuta localmente en la máquina. Para la ejecución de producción, los ejecutores Airflow o Kubeflow podrían ser más adecuados.

#### Cómo construir una canalización

In [ ]:
import os
import tempfile
from tfx import v1 as tfx

# Select a persistent TFX root directory to store your output artifacts.
# For demonstration purposes only, we use a temporary directory.
PIPELINE_ROOT = tempfile.mkdtemp()
# Select a pipeline name so that multiple runs of the same logical pipeline
# can be grouped.
PIPELINE_NAME = "function-based-pipeline"
# We use a ML Metadata configuration that uses a local SQLite database in
# the pipeline root directory. Other backends for ML Metadata are available
# for production usage.
METADATA_CONNECTION_CONFIG = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    os.path.join(PIPELINE_ROOT, 'metadata.sqlite'))

def function_based_pipeline():
  # Here, we construct our generator and consumer components in the same way.
  generator = MyGenerator()
  consumer = MyConsumer(
      data=generator.outputs['data'],
      algorithm='md5')

  return tfx.dsl.Pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      components=[generator, consumer],
      metadata_connection_config=METADATA_CONNECTION_CONFIG)

my_pipeline = function_based_pipeline()

#### Cómo ejecutar la canalización con `LocalDagRunner`

In [ ]:
tfx.orchestration.LocalDagRunner().run(my_pipeline)

Podemos inspeccionar los artefactos de salida generados por la ejecución de esta canalización.

In [ ]:
!find {PIPELINE_ROOT}

¡Ahora ha escrito sus propios componentes personalizados y ha orquestado su ejecución en LocalDagRunner! Para conocer los próximos pasos a seguir, consulte tutoriales y guías adicionales en el [sitio web de TFX](https://www.tensorflow.org/tfx).